## 4. Modeling
### 4.1 Importing modules & loading data
### 4.2 Unsupervised clustering  
### 4.3 Conclusion & summary

### 4.1 Importing modules & loading data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### 4.2 Unsupervised clustering  
#### Clustering states by population, employeed, unemployment_rate, total_licensees, total_gun_deaths, gender, age, sectors, mentor_health_issues